In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!nvidia-smi

Sat Sep 27 14:50:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Grounding DINO Demo

In [4]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-z_vcyvei
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-z_vcyvei
  Resolved https://github.com/huggingface/transformers.git to commit 071eb5334f5a9ac2c7a13515219be8a272388ec6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 9.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.57.0.dev0-py3-none-any.whl size=11510125 sha256=7bde6c860a01c6616adfcf5f244272719f9ebe44280dd340263ab71a6caaeddc
  Stored in directory: /tmp/pip-ephem-wheel-cache-65ojudfe/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df79185

**Sample BLIP**

In [5]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

def generate_caption(image_path, max_new_tokens=25, device=None):
    """
    Sinh caption từ ảnh bằng BLIP
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load model + processor
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained(
        "Salesforce/blip-image-captioning-large"
    ).to(device).eval()

    # Load ảnh
    image = Image.open(image_path).convert("RGB")

    # Tạo input cho BLIP
    inputs = processor(image, return_tensors="pt").to(device)

    # Sinh caption
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens)

    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

# ------------------ DEMO ------------------
image_path = "/kaggle/input/crop-img/Screenshot 2025-09-27 215322.png"
caption = generate_caption(image_path)
print("Caption BLIP:", caption)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Caption BLIP: a close up of a person riding a horse with a sword


In [8]:
image_path = "/kaggle/input/crop-sth/0004_clip18_frame094_14653_18.jpg"
caption = generate_caption(image_path)
print("Caption BLIP:", caption)

Caption BLIP: anime scene of a man on a horse with a crown on his head


In [7]:
image_path = "/kaggle/input/crop-sth/04_l.png"
caption = generate_caption(image_path)
print("Caption BLIP:", caption)

Caption BLIP: anime character with red hair holding a sword in a cloudy sky


In [9]:
image_path = "/kaggle/input/crop-sth/04_m.png"
caption = generate_caption(image_path)
print("Caption BLIP:", caption)

Caption BLIP: a close up of a person on a horse with a sword


In [10]:
image_path = "/kaggle/input/crop-sth/04_m_l.png"
caption = generate_caption(image_path)
print("Caption BLIP:", caption)

Caption BLIP: a close up of a person riding a horse with a sword


In [11]:
image_path = "/kaggle/input/crop-sth/0008_clip34_frame083_14653_34.jpg"
caption = generate_caption(image_path)
print("Caption BLIP:", caption)

Caption BLIP: anime image of a woman riding a horse with a sword


In [12]:
image_path = "/kaggle/input/crop-sth/08.png"
caption = generate_caption(image_path)
print("Caption BLIP:", caption)

Caption BLIP: anime image of a man riding a horse with a sword


### openmmlab-community/mm_grounding_dino_base_all

In [ ]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection

model_id = "openmmlab-community/mm_grounding_dino_large_all"
device = "cuda"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)


In [ ]:
!pip install --upgrade transformers


In [ ]:

# Check for cats and remote controls
import re, numpy as np, torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

# 1) Prompt hạt giống (rất chung, dùng được cho cartoon & real)
SEED_PROMPTS = [
    "object", "person", "animal",
    "cartoon character", "animated character", "toy",
    "electronic device", "furniture", "vehicle"
]

# Từ điển nhỏ, đủ xài cho cartoon phổ thông
COLORS  = {"red","pink","blue","green","yellow","purple","orange","black","white","gray","brown"}
ANIMALS = {"bear","rabbit","mouse","cat","dog","moose","beaver","squirrel","porcupine","skunk","fox","deer","bird"}

def gen_caption_prompts(image_pil, max_new_tokens=25):
    """Dùng BLIP để sinh caption rồi chuyển thành các prompt ngắn, tổng quát."""
    try:
        cap_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        cap_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").eval().to("cuda" if torch.cuda.is_available() else "cpu")
        inputs = cap_processor(image_pil, return_tensors="pt").to(cap_model.device)
        out = cap_model.generate(**inputs, max_new_tokens=max_new_tokens)
        caption = cap_processor.decode(out[0], skip_special_tokens=True).lower()
    except Exception:
        caption = ""  # nếu không có internet / model, vẫn chạy được với seed

    # Rút trích đơn giản: màu + danh từ khả dĩ
    words = re.findall(r"[a-z]+", caption)
    found_colors  = [w for w in words if w in COLORS]
    found_animals = [w for w in words if w in ANIMALS]
    # Một số danh mục tổng quát dễ gặp
    GENERICS = {"remote", "remote control", "tv", "screen", "phone", "laptop",
                "chair", "table", "bed", "sofa", "book", "clock", "bag", "hat", "bow"}

    found_generics = [w for w in GENERICS if w in caption]

    prompts = set()
    # màu + loài: "blue rabbit", "pink bear"
    for c in found_colors:
        for a in found_animals or ["character", "animal"]:
            prompts.add(f"{c} {a}")
            prompts.add(f"{c} cartoon {a}")
            prompts.add(f"{c} animated {a}")

    # các danh mục tổng quát phát hiện từ caption
    for g in found_generics:
        prompts.add(g)
        prompts.add(f"cartoon {g}")
        prompts.add(f"animated {g}")

    # nếu caption nói đến “cartoon/animation”, bổ sung vài cụm rộng
    if "cartoon" in caption or "animation" in caption or "animated" in caption:
        prompts.update({"cartoon animal", "cartoon face", "animated character", "mascot", "anthropomorphic animal"})

    # fallback: nếu rỗng, vẫn trả về vài cụm chung
    if not prompts:
        prompts.update({"cartoon character", "animated character", "animal", "object"})

    # Giới hạn số lượng để tránh nhiễu
    prompts = list(prompts)[:20]
    return prompts, caption

def detect_general(image_pil, processor, model, box_th=0.5, text_th=0.4, device="cuda"):
    # 2) Tạo prompt tổng: seed + auto-expansion
    auto_prompts, caption = gen_caption_prompts(image_pil)
    phrases = list(dict.fromkeys(SEED_PROMPTS + auto_prompts))[:32]  # unique + cap length

    # 3) Grounding DINO inference
    image_np = np.array(image_pil.convert("RGB"))
    inputs = processor(images=image_np, text=[phrases], return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)

    results = processor.post_process_grounded_object_detection(
        outputs,
        threshold=0.25,
        target_sizes=[image_pil.size[::-1]]
    )[0]

    # trả về cả caption để debug
    return phrases, caption, results

# ----------------- USE -----------------
# image = Image.open("/kaggle/input/cartoon/data/frames/scene_11_frame_783.jpg").convert("RGB")
# phrases, caption, result = detect_general(image, processor, model, box_th=0.35, text_th=0.25, device=device)
# print("caption:", caption)
# print("phrases:", phrases)
# for box, score, tl in zip(result["boxes"], result["scores"], result["text_labels"]):
#     print(f"{tl} {score:.2f} @ {[round(x,2) for x in box.tolist()]}")

# Load ảnh
image = Image.open("/kaggle/input/cartoon/data/frames/scene_3_frame_131.jpg").convert("RGB")

# Gọi hàm detect_general (tự sinh prompt và detect)
phrases, caption, result = detect_general(image, processor, model, box_th=0.3, text_th=0.25, device=device)

# In ra caption auto
print("BLIP caption:", caption)
print("Prompts dùng:", phrases)

# In kết quả detection
for box, score, text_label in zip(result["boxes"], result["scores"], result["text_labels"]):
    box = [round(x, 2) for x in box.tolist()]
    print(f"Detected {text_label} with confidence {round(score.item(), 3)} at location {box}")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

fig, ax = plt.subplots(1, figsize=(12, 8))
ax.imshow(image)

for box, score, text_label in zip(result["boxes"], result["scores"], result["text_labels"]):
    x1, y1, x2, y2 = box.tolist()
    rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor="red", facecolor="none")
    ax.add_patch(rect)
    ax.text(x1, y1, f"{text_label} {score:.2f}", color="yellow", fontsize=12, bbox=dict(facecolor="black", alpha=0.5))

plt.axis("off")
plt.show()


Download frame

In [ ]:
!gdown 1Y42K4JuC8L6y8YPOTiTdkudhgYaaDq3q -O AnimeRunSample.zip

In [ ]:
!unzip AnimeRunSample.zip -d AnimeRunSample/

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import zipfile

# --- Thư mục input/output ---
input_dir = "/kaggle/working/AnimeRunSample/AnimeRunSample"
output_dir = "/kaggle/working/Process/AnimeRunSample"
zip_path = "/kaggle/working/Process/AnimeRunSample.zip"

os.makedirs(output_dir, exist_ok=True)

# --- Lặp qua tất cả ảnh trong thư mục ---
for filename in os.listdir(input_dir):
    if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img_path = os.path.join(input_dir, filename)
    image = Image.open(img_path).convert("RGB")

    # Gọi detect_general cho từng ảnh
    phrases, caption, result = detect_general(
        image, processor, model,
        box_th=0.5, text_th=0.4, device=device
    )

    # Vẽ box
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)

    for box, score, text_label in zip(result["boxes"], result["scores"], result["text_labels"]):
        x1, y1, x2, y2 = box.tolist()
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=2, edgecolor="red", facecolor="none"
        )
        ax.add_patch(rect)
        ax.text(
            x1, y1,
            f"{text_label} {score:.2f}",
            color="yellow", fontsize=12,
            bbox=dict(facecolor="black", alpha=0.5)
        )

    ax.axis("off")

    # Lưu ảnh kết quả
    out_path = os.path.join(output_dir, filename)
    plt.savefig(out_path, bbox_inches="tight", pad_inches=0)
    plt.close(fig)

# --- Nén toàn bộ thư mục output thành file zip ---
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            filepath = os.path.join(root, file)
            arcname = os.path.relpath(filepath, output_dir)
            zipf.write(filepath, arcname)

print("✅ Hoàn tất! File zip ở:", zip_path)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread("/kaggle/working/AnimeRunSample/AnimeRunSample/scene_10_frame_777.jpg")
plt.imshow(img)
plt.axis("off")  # ẩn trục
plt.show()


In [ ]:
import os
import gdown
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import zipfile
from tqdm import tqdm  # để hiển thị progress bar

# --- 1. Tải video từ Google Drive ---
drive_url = "https://drive.google.com/file/d/1Ww6HASO9rOdn9N6dawv2-DNTRijpLkzE/view?usp=drive_link"
video_path = "/kaggle/working/input_video.mp4"

# Tải file (fuzzy=True để tự động nhận ID từ link view)
gdown.download(url=drive_url, output=video_path, fuzzy=True, quiet=False)

# --- 2. Chuẩn bị thư mục lưu frames ---
frames_dir = "/kaggle/working/processed_frames"
os.makedirs(frames_dir, exist_ok=True)

# Mở video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frame_idx = 0

# --- 3. Xử lý từng frame ---
with tqdm(total=total_frames, desc="Processing frames") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Chuyển BGR (OpenCV) -> RGB (PIL)
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # 🔹 Gọi detect_general (bạn phải có sẵn processor, model, device)
        phrases, caption, result = detect_general(
            image, processor, model, box_th=0.3, text_th=0.25, device=device
        )

        # Vẽ bounding box với Matplotlib
        fig, ax = plt.subplots(1, figsize=(12, 8))
        ax.imshow(image)
        for box, score, text_label in zip(result["boxes"], result["scores"], result["text_labels"]):
            x1, y1, x2, y2 = box.tolist()
            rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                     linewidth=2, edgecolor="red", facecolor="none")
            ax.add_patch(rect)
            ax.text(x1, y1, f"{text_label} {score:.2f}",
                    color="yellow", fontsize=12,
                    bbox=dict(facecolor="black", alpha=0.5))
        ax.axis("off")

        out_frame_path = os.path.join(frames_dir, f"frame_{frame_idx:05d}.jpg")
        plt.savefig(out_frame_path, bbox_inches="tight", pad_inches=0)
        plt.close(fig)

        frame_idx += 1
        pbar.update(1)

cap.release()

# --- 4. Xuất video đầu ra từ frames ---
output_video = "/kaggle/working/output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

for i in range(frame_idx):
    frame_path = os.path.join(frames_dir, f"frame_{i:05d}.jpg")
    img = cv2.imread(frame_path)
    out.write(img)

out.release()

# --- 5. Nén thư mục frame thành ZIP ---
zip_path = "/kaggle/working/processed_frames.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(frames_dir):
        for file in files:
            filepath = os.path.join(root, file)
            arcname = os.path.relpath(filepath, frames_dir)
            zipf.write(filepath, arcname)

print("✅ Hoàn tất!")
print("Video đầu ra:", output_video)
print("ZIP frames:", zip_path)


In [ ]:
import os
import zipfile
frames_dir = "/kaggle/working/processed_frames"
# --- 5. Nén thư mục frame thành ZIP ---
zip_path = "/kaggle/working/processed_frames.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(frames_dir):
        for file in files:
            filepath = os.path.join(root, file)
            arcname = os.path.relpath(filepath, frames_dir)
            zipf.write(filepath, arcname)

print("✅ Hoàn tất!")
#print("Video đầu ra:", output_video)
print("ZIP frames:", zip_path)

### IDEA-Research/grounding-dino-base

In [ ]:
"""import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection

model_id = "IDEA-Research/grounding-dino-base"
device = "cuda"

processor2 = AutoProcessor.from_pretrained(model_id)
model2 = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)"""


In [ ]:

# Check for cats and remote controls
import re, numpy as np, torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

# 1) Prompt hạt giống (rất chung, dùng được cho cartoon & real)
SEED_PROMPTS = [
    "object", "person", "animal",
    "cartoon character", "animated character", "toy",
    "electronic device", "furniture", "vehicle"
]

# Từ điển nhỏ, đủ xài cho cartoon phổ thông
COLORS  = {"red","pink","blue","green","yellow","purple","orange","black","white","gray","brown"}
ANIMALS = {"bear","rabbit","mouse","cat","dog","moose","beaver","squirrel","porcupine","skunk","fox","deer","bird"}

def gen_caption_prompts(image_pil, max_new_tokens=25):
    """Dùng BLIP để sinh caption rồi chuyển thành các prompt ngắn, tổng quát."""
    try:
        cap_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        cap_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").eval().to("cuda" if torch.cuda.is_available() else "cpu")
        inputs = cap_processor(image_pil, return_tensors="pt").to(cap_model.device)
        out = cap_model.generate(**inputs, max_new_tokens=max_new_tokens)
        caption = cap_processor.decode(out[0], skip_special_tokens=True).lower()
    except Exception:
        caption = ""  # nếu không có internet / model, vẫn chạy được với seed

    # Rút trích đơn giản: màu + danh từ khả dĩ
    words = re.findall(r"[a-z]+", caption)
    found_colors  = [w for w in words if w in COLORS]
    found_animals = [w for w in words if w in ANIMALS]
    # Một số danh mục tổng quát dễ gặp
    GENERICS = {"remote", "remote control", "tv", "screen", "phone", "laptop",
                "chair", "table", "bed", "sofa", "book", "clock", "bag", "hat", "bow"}

    found_generics = [w for w in GENERICS if w in caption]

    prompts = set()
    # màu + loài: "blue rabbit", "pink bear"
    for c in found_colors:
        for a in found_animals or ["character", "animal"]:
            prompts.add(f"{c} {a}")
            prompts.add(f"{c} cartoon {a}")
            prompts.add(f"{c} animated {a}")

    # các danh mục tổng quát phát hiện từ caption
    for g in found_generics:
        prompts.add(g)
        prompts.add(f"cartoon {g}")
        prompts.add(f"animated {g}")

    # nếu caption nói đến “cartoon/animation”, bổ sung vài cụm rộng
    if "cartoon" in caption or "animation" in caption or "animated" in caption:
        prompts.update({"cartoon animal", "cartoon face", "animated character", "mascot", "anthropomorphic animal"})

    # fallback: nếu rỗng, vẫn trả về vài cụm chung
    if not prompts:
        prompts.update({"cartoon character", "animated character", "animal", "object"})

    # Giới hạn số lượng để tránh nhiễu
    prompts = list(prompts)[:20]
    return prompts, caption

def detect_general(image_pil, processor, model, box_th=0.35, text_th=0.25, device="cuda"):
    # 2) Tạo prompt tổng: seed + auto-expansion
    auto_prompts, caption = gen_caption_prompts(image_pil)
    phrases = list(dict.fromkeys(SEED_PROMPTS + auto_prompts))[:32]  # unique + cap length

    # 3) Grounding DINO inference
    image_np = np.array(image_pil.convert("RGB"))
    inputs = processor(images=image_np, text=[phrases], return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)

    results = processor.post_process_grounded_object_detection(
        outputs,
        inputs.input_ids,
        box_threshold=box_th,
        text_threshold=text_th,
        target_sizes=[image_pil.size[::-1]]
    )[0]

    # trả về cả caption để debug
    return phrases, caption, results

# ----------------- USE -----------------
# image = Image.open("/kaggle/input/cartoon/data/frames/scene_11_frame_783.jpg").convert("RGB")
# phrases, caption, result = detect_general(image, processor, model, box_th=0.35, text_th=0.25, device=device)
# print("caption:", caption)
# print("phrases:", phrases)
# for box, score, tl in zip(result["boxes"], result["scores"], result["text_labels"]):
#     print(f"{tl} {score:.2f} @ {[round(x,2) for x in box.tolist()]}")

# Load ảnh
image2 = Image.open("/kaggle/input/cartoon/data/original/Image0182.png").convert("RGB")

# Gọi hàm detect_general (tự sinh prompt và detect)
phrases2, caption2, result2 = detect_general(image2, processor2, model2, box_th=0.3, text_th=0.25, device=device)

# In ra caption auto
print("BLIP caption:", caption2)
print("Prompts dùng:", phrases2)

# In kết quả detection
for box, score, text_label in zip(result2["boxes"], result2["scores"], result2["text_labels"]):
    box = [round(x, 2) for x in box.tolist()]
    print(f"Detected {text_label} with confidence {round(score.item(), 3)} at location {box}")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

fig, ax = plt.subplots(1, figsize=(12, 8))
ax.imshow(image2)

for box, score, text_label in zip(result2["boxes"], result2["scores"], result2["text_labels"]):
    x1, y1, x2, y2 = box.tolist()
    rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor="red", facecolor="none")
    ax.add_patch(rect)
    ax.text(x1, y1, f"{text_label} {score:.2f}", color="yellow", fontsize=12, bbox=dict(facecolor="black", alpha=0.5))

plt.axis("off")
plt.show()


# Story Coherence Evaluation

## Ý tưởng

Sử dụng **BLIP** để đánh giá độ phù hợp về mặt câu chuyện giữa:
1. **Ảnh gốc (Full Image)** - Bối cảnh tổng thể của câu chuyện
2. **Vùng crop (từ Object Detection)** - Các chi tiết cục bộ

### Pipeline

```
┌─────────────┐
│  Ảnh gốc    │
│  (Full)     │
└──────┬──────┘
       │
       ├──► BLIP Caption ──► "a cartoon character running in the forest"
       │                     (Bối cảnh tổng thể)
       │
       ▼
┌─────────────────────┐
│ Object Detection    │
│ (Grounding DINO)    │
└──────┬──────────────┘
       │
       ├──► Crop 1: Character face
       ├──► Crop 2: Background tree
       └──► Crop 3: Running shoes
       
       ▼
┌─────────────────────┐
│ BLIP Caption mỗi    │
│ vùng crop           │
└──────┬──────────────┘
       │
       ├──► Crop 1: "a smiling cartoon character"
       ├──► Crop 2: "green trees in forest"  
       └──► Crop 3: "red and white shoes"
       
       ▼
┌─────────────────────┐
│ Semantic Similarity │
│ (Sentence-BERT)     │
└──────┬──────────────┘
       │
       └──► Scores:
            - Crop 1: 0.85 ✓ (liên quan cao)
            - Crop 2: 0.72 ✓ (liên quan trung bình)
            - Crop 3: 0.45 ✗ (ít liên quan)
```

## Mục đích

### 1. **Lọc vùng quan trọng**
- Vùng có **similarity cao** → giữ được bối cảnh câu chuyện
- Vùng có **similarity thấp** → chi tiết không quan trọng

### 2. **Đánh giá chất lượng crop**
- Caption của crop có "kể được câu chuyện" không?
- Crop có capture được ý nghĩa chính không?

### 3. **Ranking crops**
Sắp xếp các vùng theo độ quan trọng:
- **Story Similarity** (mới): Phù hợp về câu chuyện
- **Detection Score**: Độ tin cậy của model
- **Edge Density**: Độ phức tạp nét vẽ
- **Entropy**: Độ phức tạp thông tin

## Ví dụ thực tế

**Ảnh gốc**: "Two friends playing basketball in the park"

| Crop | Caption | Similarity | Đánh giá |
|------|---------|-----------|----------|
| 1 | "A person holding a basketball" | 0.89 | ✓ Quan trọng |
| 2 | "Green grass and trees" | 0.65 | ~ Phụ |
| 3 | "Blue sky with clouds" | 0.42 | ✗ Ít liên quan |

→ **Crop 1** nên được ưu tiên vì giữ được câu chuyện chính!

In [ ]:
# Install required package
!pip install sentence-transformers

# Import
import sys
sys.path.append('/home/serverai/ltdoanh/LayoutGeneration')
from objectfree.story_coherence_evaluator import StoryCoherenceEvaluator

# Initialize
evaluator = StoryCoherenceEvaluator(
    blip_model_name="Salesforce/blip-image-captioning-large",
    device="cuda"
)

In [ ]:
# Test với một ảnh đơn
test_image = "/kaggle/input/keyframes/14653_keyframes/0002_clip11_frame071_14653_11.jpg"

# Giả sử đã có detection results
test_detections = [
    {'bbox': [100, 100, 300, 300], 'score': 0.9, 'label': 'character'},
    {'bbox': [400, 200, 600, 400], 'score': 0.8, 'label': 'background'},
    {'bbox': [50, 50, 150, 150], 'score': 0.7, 'label': 'detail'}
]

# Chạy evaluation
result = evaluator.evaluate_single_image(
    image_path=test_image,
    detections=test_detections,
    output_dir="/kaggle/working/story_eval",
    save_crops=True
)

# Hiển thị kết quả
print("\n" + "="*70)
print("RESULTS")
print("="*70)
print(f"Full Image Caption: {result['full_caption']}")
print(f"\nNumber of crops: {result['num_valid_crops']}")
print(f"Average similarity: {result['avg_similarity']:.3f}")
print(f"Max similarity: {result['max_similarity']:.3f}")
print(f"Min similarity: {result['min_similarity']:.3f}")

print("\n" + "-"*70)
print("TOP 3 CROPS (ranked by story similarity)")
print("-"*70)
for i, crop in enumerate(result['crops'][:3]):
    print(f"\n{i+1}. Crop {crop['crop_id']}")
    print(f"   Caption: \"{crop['crop_caption']}\"")
    print(f"   Story Similarity: {crop['story_similarity']:.3f}")
    print(f"   Detection Score: {crop['detection_score']:.3f}")
    print(f"   Label: {crop['detection_label']}")
